In [1]:
# Global variables to save the left and the right lines from a previous frame
import numpy as np
global lastR, lastL
global state

lastR=np.array((0,0,0,0))
lastL=np.array((0,0,0,0))
state=True



In [2]:
# This function is used to find the intersection points between the two lines(left & right) and the upper horizontal line(upper side of the mask) and the lower horizontal line(image hight)
def getIntersectionWithHorizontal(firstLine,y):
    import numpy as np
    if firstLine[2]==firstLine[0]:
        return np.nan
    m=(firstLine[3]-firstLine[1])/(firstLine[2]-firstLine[0])
    if m!=np.nan :
        c=firstLine[3]-m*firstLine[2]
        x=(y-c)/m
        return np.int(x)
    else:
        return np.nan


In [3]:
#Pipeline algorithm
def process_image(images):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    # The libraries that we will need
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    import numpy as np
    import cv2
    %matplotlib inline
    import math 
    
       
    
    
    # converting the image to gray scale
    imgGray=cv2.cvtColor(images,cv2.COLOR_RGB2GRAY)

    # Gaussian bluring (smoothing) with kernel 7
    kernel=7
    imgGray=cv2.GaussianBlur(imgGray,(kernel,kernel),0)

    # Edge Detection using Canny algorithm
    low_thr=100
    high_thr=200
    edges=cv2.Canny(imgGray,low_thr,high_thr)
    #plt.imshow(edges,cmap='gray')

    # mask to determine Rigion of Interest
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   
    
    imshape = images.shape
    vertices = np.array([[(20,imshape[0]),(450, 310), (480,310), (imshape[1],imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)
    #plt.imshow(masked_edges,'gray')

    # Hough transform to find lines in the gray image
    import math
    rho=1
    theta=np.pi/180
    thr=5
    min_line_length=10
    max_line_gap=5
    lines=cv2.HoughLinesP(masked_edges,rho,theta,thr,np.array([]),min_line_length,max_line_gap)
    
    # The next piece of code is used to get a right and a lift solid line by executing the following steps
    # 1- Dividing the lines from Hough transform to two groups, right and left depending on their slopes
    # 2- Determinig the largest line in each group and taking it as a reference
    # 3- Getting the intersection points between the largest line and the upper (mask upper side) and lower (image height) horizental lines 
    # 4- Getting a line between these two intersection points
    # 5- Getting the averge between the new lines and the previous ones in the previous frame
    
    line_image=np.copy(images)*0 #Image to save our lines (3 channel image)

    #Variables to be used
    largestRight=(0,0,0,0)
    largestLeft=(0,0,0,0)
    largestRightSize=0
    largestLeftSize=0
    
    #step 1 (Dividing into two groups) and step 2 (Determining the largest line)
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope=(y2-y1)/(x2-x1)
            size=math.hypot(x2-x1,y2-y1)
            if slope< -0.5:
                if size>largestLeftSize:
                    largestLeft=(x1,y1,x2,y2)
            elif slope>0.5:
                if size>largestRightSize:
                    largestRight=(x1,y1,x2,y2)


    # step 3 (Intersection points) and step 4 (getting the the two lines)
    y1=320;# upper horizontal line (y of the mask upper side, it is a little bigger value for some tuning)
    xl1=getIntersectionWithHorizontal(largestLeft,y1)+5 #Upper left point, integer 5 just for some tuning
    xr1=getIntersectionWithHorizontal(largestRight,y1)#Upper right point

    y2=imgGray.shape[0]; # Lower horizontal line (the height of the image)
    xl2=getIntersectionWithHorizontal(largestLeft,y2) #Lower left point
    xr2=getIntersectionWithHorizontal(largestRight,y2)#Lower right point

    # step 5 (Getting the averge between the new lines and the previous ones in the previous frame)
    # The value of lastR and lastL is updated every frame to be used in the next one that is why there are globally declared
    #if xl1!=np.nan and xr1!=np.nan and xl2!=np.nan and xr2!=np.nan :
    
    if state:
        lastR=[xr1,y1,xr2,y2]
        lastL=[xl1,y1,xl2,y2]
        global state
        state=False
    elif (not np.isnan(xl1)) and (not np.isnan(xr1)) and (not np.isnan(xl2)) and (not np.isnan(xr2)):
        global lastR, lastL
        lastR[0]=np.int((lastR[0]+xr1)/2)
        lastR[2]=np.int((lastR[2]+xr2)/2)
        lastR[1]=y1
        lastR[3]=y2
        lastL[0]=np.int((lastL[0]+xl1)/2)
        lastL[2]=np.int((lastL[2]+xl2)/2)
        lastL[1]=y1
        lastL[3]=y2
    
    #Drawing the lines in an image  
    cv2.line(line_image,(lastR[0],lastR[1]),(lastR[2],lastR[3]),(255,0,0),8)
    cv2.line(line_image,(lastL[0],lastL[1]),(lastL[2],lastL[3]),(255,0,0),8)
    
    #flipping the line image into the original image
    result=cv2.addWeighted(images,0.8,line_image,1,0)
    return result

<ipython-input-3-7e59ffe098a1>:94: SyntaxWarning: name 'state' is used prior to global declaration
  global state
<ipython-input-3-7e59ffe098a1>:97: SyntaxWarning: name 'lastR' is assigned to before global declaration
  global lastR, lastL
<ipython-input-3-7e59ffe098a1>:97: SyntaxWarning: name 'lastL' is assigned to before global declaration
  global lastR, lastL


In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

/opt/anaconda3/lib/python3.5/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [6]:
# The (process_image) pipeline is used to detect lane from a video
white_output = 'white.mp4'
clip1 = VideoFileClip('./solidWhiteRight.mp4')
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)



MoviePy: building video file white.mp4
----------------------------------------

Writing video into white.mp4
|----------| 0/213   0% [elapsed: 00:00 left: ?, ? iters/sec]

/opt/anaconda3/lib/python3.5/site-packages/moviepy/audio/io/readers.py:110: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.nchannels))


Done writing video in white.mp4 !
Your video is ready !
CPU times: user 31.6 s, sys: 312 ms, total: 31.9 s
Wall time: 11.3 s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))



In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))